In [6]:
from recognizer.recognizer import AzureDocument 
import os
import pickle

### Save Data

In [2]:
document = AzureDocument('pass_keys.json')

In [3]:
dir_pdfs = 'resources/sample-pdf/'
sample_pdf = os.listdir(dir_pdfs)
def repnb_f(name:str):
	repnb, _ = name.split("-")
	return repnb

full_names_pdf = [f'{dir_pdfs}{x}' for x in sample_pdf]

repnbs = [repnb_f(sample) for sample in sample_pdf]
repnbs = [rf'output_data\{x}.pkl' for x in repnbs]


for pdf, data in zip(full_names_pdf, repnbs):
	# a  = not os.path.exists(data)
	# print(a)
	if not os.path.exists(data):
		print(pdf, data)
		document.process_document(pdf, name = data)

resources/sample-pdf/PAD2429-27868850.pdf output_data\PAD2429.pkl
resources/sample-pdf/PAD2618-31156674.pdf output_data\PAD2618.pkl
resources/sample-pdf/PAD2836-30640853.pdf output_data\PAD2836.pkl
resources/sample-pdf/PAD3007-33830179.pdf output_data\PAD3007.pkl
resources/sample-pdf/PAD3350-31584078.pdf output_data\PAD3350.pkl
resources/sample-pdf/PAD3718-33702307.pdf output_data\PAD3718.pkl
resources/sample-pdf/PAD4212-33334137.pdf output_data\PAD4212.pkl
resources/sample-pdf/PAD4679-33653190.pdf output_data\PAD4679.pkl
resources/sample-pdf/PAD5244-34010307.pdf output_data\PAD5244.pkl


# Process

In [252]:
# 2017
x = 3
with open(repnbs[x], 'rb') as f:
	print(full_names_pdf[x], repnbs[x])
	a = pickle.load(f)
a['data_process'].iloc[:5, :]
# save data
all_data_pad = {}

resources/sample-pdf/PAD2836-30640853.pdf output_data\PAD2836.pkl


In [257]:
a['roles']

['pageHeader',
 'pageNumber',
 None,
 'pageFooter',
 'title',
 'footnote',
 'sectionHeading']

In [253]:
import warnings
warnings.filterwarnings('ignore')
# model 2
data = a['data_process'].query('role != "title"')
# data
h2 = ['project_description', 'implementation']
trash = ['annex']
data['lower'] = data['text_role'].str.lower()
data['h2'] = data.apply(
	lambda row: 
		row['lower'] 
		if 'project description' in row['lower'] or 'implementation' 
		in row['lower'] 
		# if  any(k in row['lower'] for k in h2) 
		# and not any(k in row['lower'] for k in trash)
		else None, axis=1
	)
# data.dropna()
data['h2'].fillna(method='ffill', inplace = True)
data
data = data.dropna()
annex = data['h2'].str.contains('annex', case=False, na = False)
implementation = data['h2'].str.contains('implementation', case=False, na = False)
project_description = data[~(annex | implementation)]

proj_desc_list = project_description['paragraph_content'].values
proj_desc_text = "\n".join(proj_desc_list)
proj_desc_text
project_description['h3'] = project_description.apply(
	lambda row: 
		row['lower'] 
		if 'project components' in row['lower'] 
		or 'project beneficiaries' in row['lower'] 
		or 'project cost and financing' in row['lower']
		# if  any(k in row['lower'] for k in h2) 
		# and not any(k in row['lower'] for k in trash)
		else None, axis=1
	)

project_description['h3'].fillna(method='ffill', inplace=True)
project_description = project_description.dropna()

components = project_description['h3'].str.contains('components', case=False)
project_components = project_description[components]

proj_comp_list = project_components.query('role != "title"')['paragraph_content'].values
proj_comp_text = '\n'.join(proj_comp_list)

_, repnb_x = repnbs[x].split('\\')
repnb_x, _ = repnb_x.split('.')

content = {
	'proj_desc': {
		'content': proj_desc_text,
	},
	'proj_comp': {'content': proj_comp_text},
}

# proj_comp_list[[0, -1]]
proj_desc_list[0], proj_desc_list[-1][-100:-1]

('II. PROJECT DESCRIPTION',
 'solution for rural roads design will be assessed for the project by an independent assessment study')

In [254]:
proj_comp_list[-1][-100:-1]

',000 was granted to the Republic of Guinea to finance preparation activities related to the project'

In [256]:
a.role

AttributeError: 'dict' object has no attribute 'role'

In [208]:
import json
json_dir = "output_json/"
with open(f'{json_dir}{repnb_x}.json', 'w') as j:
	json.dump(all_data_pad, j)